<a href="https://colab.research.google.com/github/JanEggers-hr/TalkToPDF/blob/main/TalkToPDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TalkToPDF (BETA)

*v1.1beta* - Ein Notebook, mit dem man (mit einem gespendeten API-Token) chatGPT nutzen kann, um einen Text zu analysieren - gewissermaßen mit dem Text zu chatten. 

Der Text im PDF wird mit einer Standard-Bibliothek vergleichsweise stumpf eingelesen - deshalb... 

## Nur PDF-Texte, keine Bilder!

Der Code kann derzeit keinen Text lesen, der nur abfotografiert bzw. als Grafik im Text ist. Die müsste man erst mit einer Zeichenerkennung (OCR) umwandeln - später vielleicht mal.  

## Zulässige Textlängen

Um wirklich den gesamten Text im Blick zu haben, muss die Textlänge muss unter der Grenze bleiben, die das Token-Limit des jeweiligen Modells vorgibt - das sind schätzungsweise 2-3 Seiten Text bei GPT-3.5, 5 Seiten bei GPT-4. Das deutlich größere GPT4-32k steht derzeit noch nicht zur Verfügung. 

Mit dem "Condense-Modus" kann man diese Grenze umgehen: Das Skript erstellt von allen Seiten mit GPT3.5 eine Zusammenfassung. Sobald der Knopf für den Condense-Modus geklickt wird, fängt das Programm an. Bei ersten Experimenten waren die Zusammenfassungen nur 1/3 bis 1/8 so lang - man kann also bis zu 8x so viele Seiten in ein Modell quetschen: Ein 35-Seiten-Text ging gerade noch so in die Token-Grenze von GPT-3.5. 

Aufpassen: **Chatten kann ins Geld gehen!** Das Modell hat kein Gedächtnis - man muss den bisherigen Text bei jeder neuen Anfrage komplett mit übertragen; mit den teuren GPT-4-Modellen kommen so schnell halbe Dollar je Chat zusammen. Das Eindampfen von Text ist dagegen sehr günstig: es wird mit dem günstigen GPT-3.5-Modell vielleicht einen Cent für 5-10 Seiten kosten. 

## Colabs am Leben halten 

Es wird empfohlen, eine Browser-Erweiterung wie den [Colab Automatic Clicker](https://addons.mozilla.org/en-US/firefox/addon/colab-automatic-clicker/) oder [Colab Auto Reconnect (Chrome)](https://chrome.google.com/webstore/detail/colab-auto-reconnect/ifilpgffgdbhafnaebocnofaehicbkem) zu nutzen, damit die Colab-Session offen bleibt. 

## Vorbereitungen

*Bitte einmal kurz auf die nächste Zelle klicken, um den Vorbereitungs-Code auszuführen: Einstell-Widgets erzeugen, die Library für die OpenAI-API laden.*

In [4]:
#@title
import requests
import json
import math

# ipywidgets ist schon installiert
import ipywidgets as widgets
from IPython.display import display

# Modelle und Kosten definieren
# Kosten in US-Dollar je 1000 Tokens
models_token_info = {'gpt-3.5-turbo': {
                                        'pricing': 0.002,
                                        'max_tokens': 4096
                                      },
          'gpt-4': {
                                        'pricing': 0.03,
                                        'max_tokens': 8192
                                      },
          'gpt-4-32k': {
                                        'pricing': 0.06,
                                        'max_tokens': 32768
                                      }}

textbox_max_tokens = widgets.Text(
    value='0',
    placeholder='0',
    description='Max. Token:',
)

area_system = widgets.Textarea(
    value = 'Du bist chatGPT, ein KI-Sprachsystem. Du bist freundlich \
und hilfsbereit und löst alle Aufgaben Schritt für Schritt.\n',
    rows=10,
    description = 'System:'
)

# Temperatur-Slider
slider_temperature = widgets.FloatSlider(
    value=0.2,
    min=0,
    max=1,
    step=0.1,
    description='Temperatur:',
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

# Best-of-Slider
slider_bestof = widgets.IntSlider(
    value=1,
    min=1,
    max=4,
    description='Best Of:',
    orientation='horizontal',
    readout=True
)


dropdown_model = widgets.Dropdown(
    # Nimm die oben definierte Preisliste als Basis
    options=list(models_token_info.keys()),
    value=list(models_token_info.keys())[0], # gpt-3.5 als Default
    description='Modell:',
)

textbox_stop = widgets.Text(
    value='###\n',
    placeholder="###",
    description="Stop-Token:"
)

html_warning = widgets.HTML(description = "Status:", value = "OK")

# Bisschen breiter anzeigen
textbox_max_tokens.layout.width = '200px'
dropdown_model.layout.width = '300px'
area_system.layout.width = '600px'

# Vorbereitungen für die Einstellungen sind getan - jetzt die OpenAI-Libraries
print("Widgets eingerichtet.")

# Tokenizer Tiktoken einbinden
!pip install -q tiktoken
import tiktoken
print("Tokenizer tiktoken geladen.")

# Hilfsfunktion: Token berechnen
def num_tokens_from_string(string: str) -> int:
    """Returns the number of tokens in a text string."""
    # cl100k_base ist der Tokenizer für Davinci, GPT-3 und GPT-4
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(string))
    return num_tokens

# OpenAI-API-Library einbinden
!pip install -q openai
import openai
print("OpenAI-API-Library geladen.")

# https://github.com/py-pdf/benchmarks
!pip install -q pypdfium2
import pypdfium2 as pdfium
print("PDF-Importer pypdfium geladen.")

# install bark as well as pytorch nightly to get blazing fast flash-attention
#!pip install -q git+https://github.com/suno-ai/bark.git && \
#  pip uninstall -y torch torchvision torchaudio && \
#  pip install -q --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu118
#from bark import SAMPLE_RATE, generate_audio, preload_models
#from IPython.display import Audio

#preload_models()
#print("Audio-Generator Bark geladen.")

# Funktion wird bei Veränderung ausgeführt
def update_params(change):
    global temperature
    global max_tokens
    global system_prompt
    global system_tokens
    global model
    global stoptokens
    global best_of
    temperature = slider_temperature.value
    model = dropdown_model.value
    best_of = slider_bestof.value
    # Token-Obergrenze umrechnen
    try:
        max_tokens = int(textbox_max_tokens.value)
        if max_tokens == 0:
            max_tokens = None
        else: 
            limit = models_token_info.get(model)['max_tokens']
            if max_tokens + system_tokens > limit: 
                html_warning = "<strong><em>Token-Obergrenze des Modells überschritten</em></strong>"
            else:
                html_warning = "OK"
    except ValueError:
        max_tokens = None
    textbox_max_tokens.value = f'{max_tokens}'
    system_prompt = area_system.value
    system_tokens = num_tokens_from_string(system_prompt)
    stoptokens = textbox_stop.value
    if (stoptokens == ""):
      stoptokens = None

# 1x aufrufen.
update_params(0)

# Verbinde die Widgets mit der Funktion zur Verarbeitung der Werte
textbox_max_tokens.observe(update_params, 'value')
slider_temperature.observe(update_params, 'value')
slider_bestof.observe(update_params, 'value')
area_system.observe(update_params, 'value')
textbox_stop.observe(update_params, 'value')
dropdown_model.observe(update_params, 'value')

Widgets eingerichtet.
Tokenizer tiktoken geladen.
OpenAI-API-Library geladen.
PDF-Importer pypdfium geladen.


## Einstellungen für das Modell

Basiseinstellungen: Temperatur (tendenziell niedriger als sonst, um reproduzierbare Ergebnisse zu bekommen), Modell, Stop-Token (der Text des PDF wird in Stop-Token eingefasst, um dem Modell eine Sinneinheit zu signalisieren), und das System-Prompt (die Aufgabenbeschreibung). 

In [5]:
#@title
from getpass import getpass
key_needed = True
while key_needed:
    openai.api_key = getpass("OpenAI-API-Key eingeben: ")
    try: 
        # Testweise Modelle abfragen
        models = openai.Model.list()['data']
        # Erfolg?
        print("API-Key gültig!")
        key_needed = False
    except: 
        print("Fehler bei Abfrage; ist der API-Key möglicherweise ungültig?")
previous_messages = []
previous_tokens = 0
spent_tokens = 0
spent_dollars = 0.00
pdf_tokens = 0
area_system.value = "Du beantwortetst Fragen zum Text. Du erklärst, was im Text \
zu finden ist und was in den Trainingsdaten des Sprachmodells zu finden ist."
system_prompt = area_system.value
system_tokens = num_tokens_from_string(system_prompt)

def on_chatbot_reset_clicked(button):
    global previous_messages
    global spent_tokens
    global spent_dollars
    global previous_tokens
    previous_tokens = 0 
    previous_messages = []
    spent_tokens = 0
    spent_dollars = 0.00
    chatbot_output_area.value = ''

# Die Einstellungs-Widgets anzeigen
# Setzt die globalen Variablen temperature, system_prompt, api_key, model, stoptokens
display(slider_temperature,
#        slider_bestof,
        dropdown_model,
        textbox_stop, 
        textbox_max_tokens,
        area_system)

# Hilfsfunktion : Kosten berechnen. 
def pricing(tokens):
    price = models_token_info.get(model)['pricing']
    # Kosten in Dollar zurückgeben
    return(tokens * price / 1000)

def chatbot(prompts):
    # Prompt 
    response = openai.ChatCompletion.create(
        model=model,
        messages=prompts,
        n=1,
#        best_of = best_of,
        stop=stoptokens,
        max_tokens=max_tokens,
        temperature=temperature,
        stream = True
    )
    return response

text_tokens = widgets.HTML(
    value = '<b>Verbrauchte Token</b>: 0 ($0.00)'
)

# Define the widget for displaying token usage
def update_token_usage_widget(value):
    global spent_tokens
    global spent_dollars
    spent_tokens += value
    spent_dollars += pricing(value)
    token_usage_text = f'PDF: {pdf_tokens} Token ($ {pricing(pdf_tokens):.3f}) <br>\
<b>Verbrauchte Token:</b> Diesmal {value} ($ {pricing(value):.3f}), \
insgesamt {spent_tokens} ($ {spent_dollars:.3f}) '
    text_tokens.value = token_usage_text

chatbot_output_area = widgets.HTML(
    value='',
    description='Dialog:',
    layout=widgets.Layout(width='100%')
)
# Hilfsfunktion: Token berechnen
def calculate_tokens(string: str) -> int:
    """Returns the number of tokens in a text string."""
    # cl100k_base ist der Tokenizer für Davinci, GPT-3 und GPT-4
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(string))
    return num_tokens

# Ausgaben von GPT formatieren:
# - \n in <br> umsetzen
# - Codeblöcke mit <pre><code> beginnen und abschließen

import re

def gptparse(text):
    # Codeblöcke
    pattern =  r'\`\`\`(?P<text>[^*]+)\`\`\`'
    text = re.sub(pattern, r'<code><pre>\g<text></pre></code>', text)
    return text


chatbot_output_area = widgets.HTML(
    value='',
    description='Dialog:',
    layout=widgets.Layout(width='100%')
)

# Define the function to be called when the chatbot is used
def on_chatbot_button_clicked(button):
    global chatbot_output
    global previous_tokens
    # Get the user's input and display it
    user_input = user_text.value
    user_text.value = ''
    # Generate a response from the chatbot
    chatbot_output_area.value += f'<p style="font-family: Verdana;"><b>Du</b>: {user_input}</p>'
    messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": pdf_prompt},
            *previous_messages,
            {"role": "user", "content": user_input},
        ]
    chatbot_output_area.value += '<p style="font-family: Verdana; font-style: italic;"><b>Chatbot: </b>' 
    # Anzahl von Tokens mit der User-Frage initiieren
    user_tokens = calculate_tokens(user_input)
    # Kosten für die Anfrage einpreisen
    update_token_usage_widget(user_tokens + system_tokens + pdf_tokens + previous_tokens)
    # Stream-Objekt mit der Antwort
    chatbot_response = chatbot(messages)
    collected_messages = []   # braucht man nicht zwingend
    # Iteriere über die Chunks (die Brocken )
    for chunk in chatbot_response:
        chunk_message = chunk['choices'][0]['delta']  # extract the message
        collected_messages.append(chunk_message)  # save the event response
        # Ausgabefenster: Neuen Chunk anhängen
        chatbot_output = chunk_message.get('content', '')
        chatbot_output_area.value += re.sub('\r?\n','<br>',chatbot_output)
        update_token_usage_widget(calculate_tokens(chatbot_output))
    # Stream-HTML-Block abschließen
    chatbot_output_area.value += '</p>'
    chatbot_output_area.value = gptparse(chatbot_output_area.value)
    # Antwort komplett in die Chathistorie aufnehmen
    chatbot_output = ''.join([m.get('content', '') for m in collected_messages])
#    audio_array = generate_audio(chatbot_output)
#    Audio(audio_array, rate=SAMPLE_RATE)
    previous_messages.extend([
        {"role": "user", "content": user_input},
        {"role": "assistant", "content": chatbot_output},
    ])
    # Vorigen Text in die Token-Nutzungs-Berechnung einfließen lassen
    previous_tokens += calculate_tokens(chatbot_output) + user_tokens

# Define the chatbot input and output widgets
user_text = widgets.Text(
    placeholder='...',
    description='Du:',
    layout=widgets.Layout(width='60%'),
)

# Definiere den Absenden-Button und binde ihn an on_chatbot_button_clicked
chatbot_button = widgets.Button(
    description='Absenden',
    layout=widgets.Layout(width='15%'),
)
chatbot_reset = widgets.Button(
    description = 'Reset',
    layout=widgets.Layout(width='15%')
)
chatbot_button.on_click(on_chatbot_button_clicked)
chatbot_reset.on_click(on_chatbot_reset_clicked)
# Abschicken auch durch Return in der user_text Box
user_text.on_submit(on_chatbot_button_clicked)



OpenAI-API-Key eingeben: ··········
API-Key gültig!


FloatSlider(value=0.2, description='Temperatur:', max=1.0)

Dropdown(description='Modell:', layout=Layout(width='300px'), options=('gpt-3.5-turbo', 'gpt-4', 'gpt-4-32k'),…

Text(value='###\n', description='Stop-Token:', placeholder='###')

Text(value='None', description='Max. Token:', layout=Layout(width='200px'), placeholder='0')

Textarea(value='Du beantwortetst Fragen zum Text. Du erklärst, was im Text zu finden ist und was in den Traini…

## PDF einlesen, an den Chatbot übergeben, Eingaben erwarten

Nach der Eingabe des OpenAI-API-Tokens lädt das Skript ein PDF hoch, extrahiert den Text, und ermöglicht eine Konversation darüber. 

In [6]:
#@title

from google.colab import files

condense_checkbox = widgets.ToggleButton(
    value=False,
    description='Aktiviere Condense Mode',
    layout=widgets.Layout(width='300px')
)

def activate_condense_mode(change):
    global system_prompt
    global pdf_prompt
    global pdf_tokens
    global file_upload
    global text_all 
    condense_checkbox.value = True
    condense_checkbox.disabled = True
    # Chat-Widgets verschwinden lassen
    user_text.disabled = True
    pdf_name = str(list(file_upload.keys())[0])
    pdf = pdfium.PdfDocument(file_upload[pdf_name])
    n_pages = len(pdf)
    text_all = ""
    seite_html = widgets.HTML()
    fortschritt = widgets.IntProgress(
        value=0,
        min=0,
        max=n_pages,
        description='Komprimiere: ',
        orientation='horizontal'
    )
    condense_checkbox.close()
    display(seite_html)
    display(fortschritt)
    display(text_tokens)
    # Setze das Modell vorübergehend auf GPT3.5-Turbo (aus Kostengründen)
    model = 'gpt-3.5-turbo'
    for page_num,page in enumerate(pdf):
        text_all += f"\n\nSEITE: {page_num+1}\n\nZUSAMMENFASSUNG: "
        # Load a text page helper
        textpage = page.get_textpage()
        # Extract text from the whole page and condense
        messages = [
              {"role": "system", "content": "Erstelle eine knappe Zusammenfassung des Textes."},
              {"role": "user", "content": textpage.get_text_range()}
          ]
        seite_html.value = f"Seite: {page_num+1} Zeichen: {len(textpage.get_text_range())} Token ca: {num_tokens_from_string(textpage.get_text_range())}"
        response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            n=1,
            temperature=0.0,
            stream = False
        ) 
        text_all += response['choices'][0]['message']['content'] + '\n###\n'
        # Update Kosten
        update_token_usage_widget(response["usage"]["prompt_tokens"])
        # Update Fortschrittsbalken
        fortschritt.value = page_num+1  
    # Fertig
    fortschritt.close()
    system_prompt = "Beantworte Fragen zu einem Text. Inhaltsangaben für jede Seite \
folgen. Wenn du Antworten zum Text auf einer Seite N geben sollst, antworte mit \
'GETTEXT N', um den gesamten Text lesen zu können."
    area_system.value = system_prompt
    # PDF-Volltext durch Kondensat ersetzen
    pdf_prompt = text_all
    pdf_tokens = num_tokens_from_string(text_all)
    # Token-Verbrauchsanzeige auf korrekte Werte für das PDF und 0 fürs aktuelle Prompt
    update_token_usage_widget(0)
    model = dropdown_model.value
    user_text.disabled = False
    display(chatbot_output_area, user_text, chatbot_button, chatbot_reset)

# Activate ausführen, sobald Checkbox geklickt wird
condense_checkbox.observe(activate_condense_mode, names='value')

def import_pdf(file_upload):
  global pdf_prompt
  # Namen des hochgeladenen PDF extrahieren
  pdf_name = str(list(file_upload.keys())[0])
  pdf = pdfium.PdfDocument(file_upload[pdf_name])
  # n_pages = len(pdf)
  # Das ist erst mal ganz stumpf: Iteriere durch alle Seiten und hänge sie 
  # an einen großen Python-String. 
  #
  # Verfeinerte Strukturierung - Seitenzahlen oä - später. 
  text_all = ""
  for page_num,page in enumerate(pdf):
      text_all += f"\n\nSEITE: {page_num+1}\n\n"
      # Load a text page helper
      textpage = page.get_textpage()
      # Extract text from the whole page
      text_all += textpage.get_text_range()
  # Update Kosten
  return(text_all)

file_upload = files.upload()
text_all = import_pdf(file_upload)
pdf_tokens = num_tokens_from_string(text_all)
print(f"{len(text_all)} Zeichen - in Token: {pdf_tokens}")
# PDF länger als Token-Fenster plus Reserven?
if pdf_tokens + system_tokens + 200 > models_token_info.get(model)['max_tokens']:
    print("Token-Fenster zu klein; Condense-Modus erforderlich.")
    print("Dieser Modus erstellt Zusammenfassungen aller Seiten und arbeitet damit.")
    print("Ein Klick auf den Button startet die Zusammenfassungen (mit gpt-3.5)")
    # Legt gleichzeitig die Checkbox lahm. 
    # Chat-Widgets verschwinden lassen
    force_condense_mode = True
else: 
    force_condense_mode = False
display(condense_checkbox)


pdf_prompt = "###\nAnalysiere diesen Text:\n" + text_all + "\n###\n"
# audio_array = generate_audio(system_prompt)
# Audio(audio_array, rate=SAMPLE_RATE)


# Die eigentliche Chatbot-Funktion findet sich in der Funktion
# on_chatbot_button_clicked()

# Display the chatbot widgets - wenn Condense Mode nicht nötig
if (not force_condense_mode):
    display(html_warning)
    display(text_tokens)
    display(chatbot_output_area, user_text, chatbot_button, chatbot_reset)


Saving RISJ paper_SimonE_TT22_Final.pdf to RISJ paper_SimonE_TT22_Final.pdf
67147 Zeichen - in Token: 13551
Token-Fenster zu klein; Condense-Modus erforderlich.
Dieser Modus erstellt Zusammenfassungen aller Seiten und arbeitet damit.
Ein Klick auf den Button startet die Zusammenfassungen (mit gpt-3.5)


ToggleButton(value=False, description='Aktiviere Condense Mode', layout=Layout(width='300px'))

HTML(value='')

IntProgress(value=0, description='Komprimiere: ', max=34)

HTML(value='<b>Verbrauchte Token</b>: 0 ($0.00)')

## Bekannte Probleme

- Manchmal verschluckt das Eingabefeld die letzten 1, 2 Zeichen
- Reset-Button funktioniert nicht (sollte Neuladen triggern)
- Obergrenze des Modells werden derzeit nicht angezeigt
- Token-Berechnung beruht auf dem Standardmodell und ist deshalb ungenau

## Nice-to-have: Verbesserungen

- Zeichen-/Token-Angabe je Seite beim Condense (Hilft auch, leere Dokumente zu identifizieren) 
- Inhaltsverzeichnis des PDF generieren und ebenfalls übergeben
- Den "Zusammenfassungsmodus" um die Möglichkeit ergänzen, dem Modell den Volltext einer Seite zur Verfügung zu stellen - über ```GETPAGE n``` 
- Codeblöcke korrekt formatieren
- Liste der Modelle auf die beschränken, die für das API-Token freigegeben sind
- Modellgenaue Token-Abrechnung

## Versionshistorie

- **V1.1beta**: Token-Verbrauchsangabe für die Zusammenfassung korrigiert; jetzt kann man die Kosten des PDF-Kondensats einigermaßen abschätzen. 
- **V1.1alpha**: Experimenteller "Condense-Modus" erstellt Zusammenfassungen für jede Seite und arbeitet mit diesen.
- **V1.0**: Funktionsfähig für Dokumente unterhalb der Token-Grenze des Modells; Streaming (stückweise Rückgabe der Ergebnisse wie bei ChatGPT) ermöglicht
- **V0.1**: Erste Funktionsdemo für PDF-Extraktion und Chat-Completion